## Accelerating M20 Calculation

Notebook that focuses on accelerating my calculation for M20, which is currently incredibly slow.

In [1]:
import pandas as pd
import numpy as np
import cv2 as cv
from shapely.geometry import Polygon, Point

from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
import astropy.units as u
from astropy.coordinates import SkyCoord

In [2]:
df_ginis = pd.read_csv(r'C:\Users\oryan\Documents\galaxy-zoo-desi\results/ginis-major.csv', index_col = 0)

In [3]:
df_ginis

,im_paths,id_str,ra,dec,petro_theta,est_petro_th50,petro_th50,petro_th90,gini,category
0,E:/GZ-DESI/images/major-dist\100012_421-cutout...,100012_421,36.303316,-44.348649,NaN,2.393999,NaN,NaN,0.41877005570873455,major
1,E:/GZ-DESI/images/major-dist\100025_3459-cutou...,100025_3459,40.747850,-44.142658,NaN,2.026541,NaN,NaN,0.47764445919663906,major
2,E:/GZ-DESI/images/major-dist\100049_651-cutout...,100049_651,49.191266,-44.344255,NaN,2.382727,NaN,NaN,0.46975089652240526,major
3,E:/GZ-DESI/images/major-dist\100061_265-cutout...,100061_265,53.409491,-44.357158,NaN,2.103341,NaN,NaN,0.4185728701900139,major
4,E:/GZ-DESI/images/major-dist\100108_3617-cutou...,100108_3617,69.833979,-44.193233,NaN,3.622461,NaN,NaN,0.2901572380610142,major
...,...,...,...,...,...,...,...,...,...,...
11589,E:/GZ-DESI/images/major-dist\99801_4400-cutout...,99801_4400,322.647672,-44.436758,NaN,9.058787,NaN,NaN,0.3759675322154746,major
11590,E:/GZ-DESI/images/major-dist\99836_4996-cutout...,99836_4996,335.028729,-44.457437,NaN,2.841919,NaN,NaN,0.3154597488854913,major
11591,E:/GZ-DESI/images/major-dist\99850_3747-cutout...,99850_3747,339.845061,-44.488863,NaN,2.081201,NaN,NaN,0.43261384737659764,major
11592,E:/GZ-DESI/images/major-dist\99857_2264-cutout...,99857_2264,342.304118,-44.498664,NaN,2.840653,NaN,NaN,0.45981817613647374,major


In [4]:
df_part_rem = df_ginis.query('gini != "partial-overlap"')
df_rem = df_part_rem.query('gini != "empty-image"')

In [5]:
row = df_rem.iloc[-1]

path = row.im_paths
petro_50 = row.est_petro_th50
ra = row.ra
dec = row.dec

In [6]:
def getting_correct_contours(contours):
    length = 0
        
    for i in contours:
        if len(i) > length:
            correct_contours = i
            length = len(i)
    
    return correct_contours

In [7]:
def conts_to_list(contours):
    contour_list = []
    for i in range(len(contours)):
        row = contours[i][0]
        contour_list.append([row[0], row[1]])
    return contour_list

In [8]:
%%time
if np.isnan(petro_50):
    print('Failed')
    sys.exit()

data = fits.getdata(path)
header = fits.getheader(path)

w = WCS(header, naxis = 2)

size = u.Quantity((4*petro_50, 4*petro_50), u.arcsec)
coord = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'icrs')
try:
    cutout = Cutout2D(data[1,:,:], coord, size, wcs = w, mode='strict')
except:
    print('partial-overlap')
    sys.exit()

if np.sum(cutout.data) == 0:
    print('empty-image')
    sys.exit()

cutout_int = cutout.data.copy()

cut = np.percentile(cutout.data,65)
cutout_int[cutout_int <= cut] = 0
cutout_int[cutout_int > cut] = 1
cutout_int = cutout_int.astype(int)

contours, _ = cv.findContours(cutout_int, cv.RETR_FLOODFILL, cv.CHAIN_APPROX_NONE)

contours_nested_list = getting_correct_contours(contours)

extracted_contour_list = conts_to_list(contours_nested_list)

contour_arr = np.zeros([len(extracted_contour_list),2])
for i in range(len(extracted_contour_list)):
    contour_arr[i,0] = extracted_contour_list[i][0]
    contour_arr[i,1] = extracted_contour_list[i][1]

pl = Polygon(contour_arr)

pixels_mask = np.zeros(cutout.data.shape).astype(bool)
for i in range(cutout.data.shape[0]):
    for j in range(cutout.data.shape[1]):
        pt = Point(i,j)
        if pl.contains(pt):
            pixels_mask[i,j] = True
pixels_mask = pixels_mask.T

gal_pixels_list = np.argwhere(pixels_mask).tolist()
gal_pixels_arr = np.asarray(gal_pixels_list)

mtot = np.inf
for i in gal_pixels_list:
    
    mtmp = np.sum(cutout.data[gal_pixels_arr[:,0], gal_pixels_arr[:,1]] * ((gal_pixels_arr[:,0] - i[0])**2 + (gal_pixels_arr[:,1] - i[1])**2))
    
    if mtmp < mtot:
        mtot = mtmp.copy()
        center = i.copy()

f_tot = np.sum(cutout.data[pixels_mask])

sum_f = 0
cutout_array = cutout.data.copy()
cutout_array[np.invert(pixels_mask)] = 0
pixels = []

while sum_f < 0.20 * f_tot:
    arr_max = np.max(cutout_array)
    indices = np.where(cutout_array == arr_max)
    x = indices[0][0]
    y = indices[1][0]
    
    pixels.append([x,y])
    
    sum_f += arr_max
    cutout_array[x,y] = 0
    
m_i = []
for i in pixels:
    x = i[0]
    y = i[1]
    f = cutout.data[x,y]
    
    m_i.append(f * ((x - center[0])**2 + (y - center[1])**2))
    
m_20 = np.log10(np.sum(m_i) / mtot)

Wall time: 170 ms


### Writing Faster MTot

In [9]:
mtot_arr = np.zeros(cutout.shape)

In [10]:
def calc_mtot_value(value, cen_x, cen_y, pixels_mask):
    m = 0
    for p in range(pixels_mask.shape[0]):
        for q in range(pixels_mask.shape[1]):
            if not pixels_mask[p,q]:
                continue
            m += value * ((p - i)**2 + (q - j)**2)
    return m

In [56]:
gal_pixels_list = np.argwhere(pixels_mask).tolist()
gal_pixels_arr = np.asarray(gal_pixels_list)

In [62]:
%%time
m_tot = np.inf
for i in gal_pixels_list:
    
    mtmp = np.sum(cutout.data[gal_pixels_arr[:,0], gal_pixels_arr[:,1]] * ((gal_pixels_arr[:,0] - i[0])**2 + (gal_pixels_arr[:,1] - i[1])**2))
    
    if mtmp < m_tot:
        m_tot = mtmp.copy()
        center = i.copy()

Wall time: 29.7 ms


In [54]:
m_20

-1.3687265313437669

In [65]:
((((168e-3) * 187000) / 60) / 60)

8.726666666666667